## 📊 논문 기반 모델 세부사항 정리

| 항목 | 설정값 / 설명 |
|------|----------------|
| **입력 이미지 크기** | 128 × 128 (Grayscale, 1채널) |
| **모델 구조** | LeNet-5 모델 3개 병렬 구성 후 Concatenation |
| **Conv1** | 6 filters, kernel size (5×5), ReLU, padding=same |
| **Pooling1** | Average Pooling (2×2) |
| **Conv2** | 16 filters, kernel size (5×5), ReLU |
| **Pooling2** | Average Pooling (2×2) |
| **FC1 (각 LeNet)** | 120 neurons, ReLU |
| **FC2 (각 LeNet)** | 84 neurons, ReLU |
| **출력층 (각 LeNet)** | 2 neurons, Softmax |
| **합치기 방식** | 세 개의 LeNet 출력 → Concatenate → Reshape(nets, 2) → 최종 Softmax |
| **총 모델 수** | 3 (병렬 LeNet-5) |
| **Optimizer** | Adam |
| **Loss Function** | Categorical Crossentropy |
| **Activation Function** | Modified ReLU |
| **정규화 기법** | Batch Normalization (각 Conv 뒤에 적용) |
| **Dropout** | ❌ 사용 여부 언급 없음 (논문 미기재) |
| **Epoch 수** | ❌ 명시되지 않음 (그래프 기반 추정 필요) |
| **데이터 분할** | Train: 4684, Validation: 1152, Test: 20 |
| **프레임워크** | Keras 2.6.0, TensorFlow 2.14.0 (Google Colab T4 GPU 사용) |
| **정확도 (논문 기준)** | Train: 99%, Test: 96% |


---
### 📊 Validation Accuracy 안정화

| 현상 | 의미 | 해석 |
|------|------|------|
| `train acc ↑`, `val acc ↑` | 학습도 잘 되고, 일반화도 잘 되고 있음 | 🔥 Best case |
| `train acc ↑`, `val acc` 요동 → 이후 안정화 | 점차 일반화되는 학습 흐름 | 👍 긍정적 흐름 |
| `train acc ↑`, `val acc ↓ (지속)` | 과적합 발생 | ⚠️ Early stopping 또는 regularization 필요 |
| `val acc ↑↓ 반복`, `train acc`도 불안정 | 학습률, 초기 가중치, 데이터 품질 문제 가능성 | 🛠️ 튜닝 필요 |

#### ✅ 결론
- Validation accuracy가 **안정화되면 모델이 일반적인 패턴을 잘 학습하고 있다는 신호**
- 초기의 출렁임은 흔한 일이며, **이후 수렴 여부**가 더 중요


---
### 🧠 Dropout이란?

**Dropout**은 신경망 학습 과정에서 일부 뉴런을 무작위로 꺼서(overfitting을 막는) 정규화 기법

> 학습 시 일부 뉴런을 무작위로 제거함으로써, 특정 경로에 의존하지 않고 일반화된 모델을 만들 수 있게 함

#### 🔍 사용 이유

- 모델이 학습 데이터에 너무 **과적합(overfitting)** 되는 것을 방지
- 다양한 뉴런 조합을 학습하게 하여 **robust한 성능** 유도
- 테스트 시에는 모든 뉴런을 사용하여 **안정적인 예측** 수행

#### 🧪 PyTorch 코드 예시

```python
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.5)  # 50% 확률로 뉴런 끔
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
```
